# Finne høyfrekvente stasjoner

Vi ønsker å finne stasjoner/holdeplasser som tilfredsstiller ulike definisjoner av høyfrekvent.

In [1]:
import polars as pl
import polars.selectors as cs

### Data fra Entur

Tre tabeller er lastet ned fra Enturs datakatalog. SQL-koden er gjengitt under. 


Vi ser kun på en dato, nemlig 2026-01-15.

Koden under gir en rutetabell for Norge torsdag 15. januar 2026. 

```sql
SELECT 
    agency_name,
    route_id, route_short_name, route_long_name, route_type, 
    trip_id, trip_headsign, direction_id, shape_dist_traveled,
    stop_id, stop_sequence, 
    DATETIME(departure_time, "Europe/Oslo") AS departure_time_oslo,
    DATETIME(arrival_time, "Europe/Oslo") AS arrival_time_oslo
FROM `ent-data-sharing-ext-prd.timetable_gtfs.gtfs_last_recorded_ent_v1`
WHERE operating_date = '2026-01-15'
```

In [ ]:
# resultatet fra koden over er lagret bq-results-20260206-080556-1770365220476.csv
alle_avganger = (
    pl.scan_csv("bq-results-20260206-080556-1770365220476.csv")
    .with_columns(
        pl.col("departure_time_oslo").str.to_datetime(format="%Y-%m-%d %H:%M:%S", time_zone="Europe/Oslo"),
        pl.col("arrival_time_oslo").str.to_datetime(format="%Y-%m-%d %H:%M:%S", time_zone="Europe/Oslo")
    )
    .rename({"departure_time_oslo":"departure_time", "arrival_time_oslo":"arrival_time"})
)

print(f"Det er {alle_avganger.select(pl.col("stop_id").len()).collect().item():,} antall rader i rutetabellen.\nSer sånn ut:")
alle_avganger.head(3).collect()

Det er 1,838,090 antall rader i rutetabellen.
Ser sånn ut


agency_name,route_id,route_short_name,route_long_name,route_type,trip_id,trip_headsign,direction_id,shape_dist_traveled,stop_id,stop_sequence,departure_time,arrival_time
str,str,i64,str,str,str,str,i64,i64,str,i64,"datetime[μs, Europe/Oslo]","datetime[μs, Europe/Oslo]"
"""Østfold kollektivtrafikk""","""OST:Line:1_477""",477,"""Mysen vgs.""","""Bus""","""OST:ServiceJourney:477_2511110…","""Trøgstad-Krokedal snuplass""",1,11140,"""NSR:Quay:100103""",10,2026-01-15 14:14:00 CET,2026-01-15 14:14:00 CET
"""Østfold kollektivtrafikk""","""OST:Line:1_206""",206,"""Krossern-Orkerød""","""Bus""","""OST:ServiceJourney:206_2511110…","""Moss sentrum""",1,1308,"""NSR:Quay:100110""",3,2026-01-15 18:17:00 CET,2026-01-15 18:17:00 CET
"""Østfold kollektivtrafikk""","""OST:Line:1_206""",206,"""Krossern-Orkerød""","""Bus""","""OST:ServiceJourney:206_2511110…","""Moss sentrum""",1,1308,"""NSR:Quay:100110""",3,2026-01-15 17:47:00 CET,2026-01-15 17:47:00 CET


```sql
SELECT 
    id, version, publicCode, transportMode, name, shortName, 
    description, location_longitude, location_latitude, 
    parentRef.ref AS parentRef_ref, parentRef.version AS parentRef_version 
FROM 
    `ent-data-sharing-ext-prd.national_stop_registry.stop_places_all_versions
````

In [53]:
stoppesteder = (
    pl.scan_csv("bq-results-20260206-082744-1770366470007.csv")
    .filter(pl.col("version")==pl.col("version").max().over("id"))
) 
print(stoppesteder.select(pl.col("id").len().alias("nrow"),pl.col("id").unique().len().alias("n_unique_id")).collect())
stoppesteder.head(3).collect()

shape: (1, 2)
┌───────┬─────────────┐
│ nrow  ┆ n_unique_id │
│ ---   ┆ ---         │
│ u32   ┆ u32         │
╞═══════╪═════════════╡
│ 64438 ┆ 64438       │
└───────┴─────────────┘


id,version,publicCode,transportMode,name,shortName,description,location_longitude,location_latitude,parentRef_ref,parentRef_version
str,i64,str,str,str,str,str,f64,f64,str,i64
"""NSR:StopPlace:6459""",27,null,"""BUS""","""Heimdalsgata""",null,"""i Trondheimsveien""",10.760974,59.918394,"""NSR:StopPlace:58253""",20
"""NSR:StopPlace:5899""",7,null,"""BUS""","""Vestbyveien""",null,"""i Bekkenstenveien""",10.887819,59.953365,"""NSR:StopPlace:59645""",6
"""NSR:StopPlace:7215""",4,null,"""BUS""","""Espa E6""",null,"""mot Oslo""",11.25553,60.563193,null,null


In [52]:
stoppesteder.group_by("parentRef_ref").agg(pl.col("id").unique()).filter(pl.col("id").list.contains(pl.col("parentRef_ref"))).collect()

parentRef_ref,id
str,list[str]


In [31]:
stoppesteder = (
    stoppesteder
    .with_columns(
        pl.when(pl.col("parentRef_ref").is_not_null()).then("parentRef_ref").otherwise("id").alias("parent_station")
    )
)
stoppesteder.collect().head()

id,version,publicCode,transportMode,name,shortName,description,location_longitude,location_latitude,parentRef_ref,parentRef_version,parent_station
str,i64,str,str,str,str,str,f64,f64,str,i64,str
"""NSR:StopPlace:60251""",2,null,"""BUS""","""Lübeck Beim Retteich""",null,"""ved ZOB/Hauptbahnhof""",10.669071,53.866164,null,null,"""NSR:StopPlace:60251"""
"""NSR:StopPlace:5357""",18,null,"""BUS""","""Oslo Lufthavn""",null,"""Ankomst, nedre plan""",11.097254,60.193455,"""NSR:StopPlace:58211""",3,"""NSR:StopPlace:58211"""
"""NSR:StopPlace:57777""",3,null,"""RAIL""","""Riksgrensen stasjon""",null,"""svensk side""",18.120886,68.426673,null,null,"""NSR:StopPlace:57777"""
"""NSR:StopPlace:59627""",6,null,"""BUS""","""Brandstad""",null,"""i Dronningveien""",10.749161,59.541712,"""NSR:StopPlace:59628""",2,"""NSR:StopPlace:59628"""
"""NSR:StopPlace:498""",3,null,"""BUS""","""Blommenholm""",null,"""(Alternativ transport)""",10.554514,59.89696,null,null,"""NSR:StopPlace:498"""


In [ ]:
stoppesteder.filter(pl.col("name").str.contains("Helsfyr$")).collect()

stoppesteder.select(pl.col("parentRef_ref").unique()).collect().join(
    
)

parentRef_ref
str
"""NSR:StopPlace:61129"""
"""NSR:StopPlace:61860"""
"""NSR:StopPlace:63364"""
"""NSR:StopPlace:60783"""
"""NSR:StopPlace:60320"""
…
"""NSR:StopPlace:63387"""
"""NSR:StopPlace:60965"""
"""NSR:StopPlace:59767"""


``` sql
SELECT 
    id, version, publicCode, name, description, 
    location_longitude, location_latitude, stopPlaceRef, 
    stopPlaceVersion 
FROM 
    `ent-data-sharing-ext-prd.national_stop_registry.quays_all_versions`
```

In [54]:
quays = (
    pl.scan_csv("bq-results-20260206-082949-1770366598757.csv")
    .filter(pl.col("version")==pl.col("version").max().over("id"))
)
quays.head(3).collect()

id,version,publicCode,name,description,location_longitude,location_latitude,stopPlaceRef,stopPlaceVersion
str,i64,str,str,str,f64,f64,str,i64
"""NSR:Quay:1""",36,"""""",null,null,10.75525,59.909548,"""NSR:StopPlace:2""",36
"""NSR:Quay:1000""",17,"""4""",null,null,7.987168,58.14577,"""NSR:StopPlace:609""",17
"""NSR:Quay:100005""",15,"""1""",null,null,15.211521,59.278877,"""NSR:StopPlace:570""",16


In [66]:
quays.filter(pl.col("id").str.contains("Station")).collect()

id,version,publicCode,name,description,location_longitude,location_latitude,stopPlaceRef,stopPlaceVersion
str,i64,str,str,str,f64,f64,str,i64


In [64]:
helsfyr = quays.filter(
    pl.col("stopPlaceRef").is_in(
        stoppesteder.filter(pl.col("name").str.contains("Helsfyr")).filter(pl.col("id")!=pl.col("parentRef_ref")).collect().get_column("id").implode()
    )
).collect()

import geopandas as gpd
from lonboard import viz

print(helsfyr.shape)

viz(gpd.GeoDataFrame(helsfyr.to_pandas(), geometry=gpd.points_from_xy(helsfyr.to_pandas()["location_longitude"], helsfyr.to_pandas()["location_latitude"]), crs=4326))

(19, 9)


### Se på ulike definisjoner

#### Høyfrekvente ruter

Bussrute hvert 10. minutt. Andre ruter hvert 15. 

##### 7-20

In [5]:
start_time = pl.datetime(2026, 1, 15, 7, time_zone="Europe/Oslo")
end_time = pl.datetime(2026, 1, 15, 20, time_zone="Europe/Oslo")

timedelta_bus = pl.duration(minutes=10)
timedelta_not_bus = pl.duration(minutes=15)

parent_station_7_20 = (
    alle_avganger
    .filter(pl.col("route_type")!="Ferry")
    .filter(pl.col("stop_sequence")!=pl.col("stop_sequence").max().over("trip_id"))
    .sort("departure_time")
    .with_columns(
        waiting_time = pl.col("departure_time").shift(-1).over("stop_id") - pl.col("departure_time")
    )
    .group_by(
        "stop_id", "parent_station", "route_type"
    )
    .agg(
        max_waiting_time_bus = pl.col("waiting_time").filter(
                pl.col("departure_time")
                .is_between(
                    start_time,
                    end_time-timedelta_bus
                )
        ).max(),
        max_waiting_time_not_bus = pl.col("waiting_time").filter(
                pl.col("departure_time")
                .is_between(
                    start_time,
                    end_time-timedelta_not_bus
                )
        ).max(),
        first_departure = pl.col("departure_time").filter(pl.col("departure_time")>=start_time).min(),
        last_departure = pl.col("departure_time").filter(pl.col("departure_time")<=end_time).max()
    )
    .with_columns(
        pl.when(pl.col("route_type")=="Bus").then("max_waiting_time_bus").otherwise("max_waiting_time_not_bus").alias("max_waiting_time")
    )
    .drop(cs.contains("bus"))
    .filter(
        (
            (pl.col("route_type")=="Bus") & 
            (pl.col("max_waiting_time")<=timedelta_bus) & 
            (pl.col("first_departure")<=start_time + timedelta_bus) & 
            (pl.col("last_departure")>=end_time - timedelta_bus)
        ) |
        (
            (pl.col("route_type")!="Bus") & 
            (pl.col("max_waiting_time")<=timedelta_not_bus) & 
            (pl.col("first_departure")<=start_time + timedelta_not_bus) & 
            (pl.col("last_departure")>=end_time - timedelta_not_bus)
        )
    )
    .group_by("parent_station", "route_type")
    .agg(pl.struct("stop_id", "first_departure", "last_departure", "max_waiting_time"))
    .collect()
    .join(
        stoppesteder.select("name", "id", cs.contains("location")), left_on="parent_station", right_on="id", how="left"
    )
)

parent_station_7_20

parent_station,route_type,stop_id,name,location_longitude,location_latitude
str,str,list[struct[4]],str,f64,f64
"""NSR:StopPlace:43133""","""Bus""","[{""NSR:Quay:73976"",2026-01-15 07:00:00 CET,2026-01-15 20:00:00 CET,9m}, {""NSR:Quay:73975"",2026-01-15 07:03:00 CET,2026-01-15 20:00:00 CET,6m}]","""Solsiden""",10.413246,63.434019
"""NSR:StopPlace:5824""","""Bus""","[{""NSR:Quay:10683"",2026-01-15 07:05:00 CET,2026-01-15 19:57:00 CET,10m}]","""Sagstuveien""",10.880565,59.954149
"""NSR:StopPlace:43421""","""Bus""","[{""NSR:Quay:74502"",2026-01-15 07:01:00 CET,2026-01-15 19:56:00 CET,10m}]","""Kvenildsmyra""",10.369216,63.335205
"""NSR:StopPlace:5397""","""Bus""","[{""NSR:Quay:9869"",2026-01-15 07:00:00 CET,2026-01-15 19:56:00 CET,9m}, {""NSR:Quay:9870"",2026-01-15 07:05:00 CET,2026-01-15 19:56:00 CET,10m}]","""Knatten""",10.96396,59.928184
"""NSR:StopPlace:32368""","""Bus""","[{""NSR:Quay:55840"",2026-01-15 07:00:00 CET,2026-01-15 20:00:00 CET,10m}]","""Torgny Segerstedts vei""",5.285415,60.343243
…,…,…,…,…,…
"""NSR:StopPlace:6024""","""Tram""","[{""NSR:Quay:11057"",2026-01-15 07:04:00 CET,2026-01-15 19:57:00 CET,13m}, {""NSR:Quay:11058"",2026-01-15 07:13:00 CET,2026-01-15 19:53:00 CET,10m}]","""Sinsenterrassen""",10.779141,59.934152
"""NSR:StopPlace:4293""","""Tram""","[{""NSR:Quay:7753"",2026-01-15 07:02:00 CET,2026-01-15 19:56:00 CET,6m}, {""NSR:Quay:7754"",2026-01-15 07:04:00 CET,2026-01-15 19:55:00 CET,7m}]","""Niels Juels gate""",10.715157,59.91634
"""NSR:StopPlace:28564""","""Bus""","[{""NSR:Quay:49076"",2026-01-15 07:02:00 CET,2026-01-15 19:58:00 CET,5m}]","""Gausel sentrum""",5.728524,58.908707


### 7-18

In [6]:
start_time = pl.datetime(2026, 1, 15, 7, time_zone="Europe/Oslo")
end_time = pl.datetime(2026, 1, 15, 18, time_zone="Europe/Oslo")
timedelta_bus = pl.duration(minutes=10)
timedelta_not_bus = pl.duration(minutes=15)

parent_station_7_18 = (
    alle_avganger
    .filter(pl.col("route_type")!="Ferry")
    .filter(pl.col("stop_sequence")!=pl.col("stop_sequence").max().over("trip_id"))
    .sort("departure_time")
    .with_columns(
        waiting_time = pl.col("departure_time").shift(-1).over("stop_id") - pl.col("departure_time")
    )
    .group_by(
        "stop_id", "parent_station", "route_type"
    )
    .agg(
        max_waiting_time_bus = pl.col("waiting_time").filter(
                pl.col("departure_time")
                .is_between(
                    start_time,
                    end_time-timedelta_bus
                )
        ).max(),
        max_waiting_time_not_bus = pl.col("waiting_time").filter(
                pl.col("departure_time")
                .is_between(
                    start_time,
                    end_time-timedelta_not_bus
                )
        ).max(),
        first_departure = pl.col("departure_time").filter(pl.col("departure_time")>=start_time).min(),
        last_departure = pl.col("departure_time").filter(pl.col("departure_time")<=end_time).max()
    )
    .with_columns(
        pl.when(pl.col("route_type")=="Bus").then("max_waiting_time_bus").otherwise("max_waiting_time_not_bus").alias("max_waiting_time")
    )
    .drop(cs.contains("bus"))
    .filter(
        (
            (pl.col("route_type")=="Bus") & 
            (pl.col("max_waiting_time")<=timedelta_bus) & 
            (pl.col("first_departure")<=start_time + timedelta_bus) & 
            (pl.col("last_departure")>=end_time - timedelta_bus)
        ) |
        (
            (pl.col("route_type")!="Bus") & 
            (pl.col("max_waiting_time")<=timedelta_not_bus) & 
            (pl.col("first_departure")<=start_time + timedelta_not_bus) & 
            (pl.col("last_departure")>=end_time - timedelta_not_bus)
        )
    )
    .group_by("parent_station", "route_type")
    .agg(pl.struct("stop_id", "first_departure", "last_departure", "max_waiting_time"))
    .collect()
    .join(
        stoppesteder.select("name", "id", cs.contains("location")), left_on="parent_station", right_on="id", how="left"
    )
)

parent_station_7_18

parent_station,route_type,stop_id,name,location_longitude,location_latitude
str,str,list[struct[4]],str,f64,f64
"""NSR:StopPlace:16821""","""Bus""","[{""NSR:Quay:29344"",2026-01-15 07:05:00 CET,2026-01-15 17:55:00 CET,10m}, {""NSR:Quay:29347"",2026-01-15 07:03:00 CET,2026-01-15 17:58:00 CET,10m}]","""Knoffs gate""",10.207058,59.736685
"""NSR:StopPlace:5607""","""Subway""","[{""NSR:Quay:10270"",2026-01-15 07:02:00 CET,2026-01-15 17:55:00 CET,8m}, {""NSR:Quay:10269"",2026-01-15 07:06:00 CET,2026-01-15 17:58:00 CET,8m}]","""Lindeberg""",10.882135,59.932874
"""NSR:StopPlace:63099""","""Bus""","[{""NSR:Quay:108875"",2026-01-15 07:00:00 CET,2026-01-15 18:00:00 CET,8m}]","""SUS Ullandhaug""",5.703546,58.929954
"""NSR:StopPlace:60""","""Train""","[{""NSR:Quay:95"",2026-01-15 07:13:00 CET,2026-01-15 17:58:00 CET,15m}, {""NSR:Quay:94"",2026-01-15 07:01:00 CET,2026-01-15 17:46:00 CET,15m}]","""Rosenholm stasjon""",10.797858,59.823624
"""NSR:StopPlace:6664""","""Bus""","[{""NSR:Quay:12307"",2026-01-15 07:02:00 CET,2026-01-15 17:57:00 CET,10m}]","""Gladvollveien""",10.778732,59.848201
…,…,…,…,…,…
"""NSR:StopPlace:6318""","""Subway""","[{""NSR:Quay:11605"",2026-01-15 07:07:00 CET,2026-01-15 17:52:00 CET,15m}, {""NSR:Quay:11606"",2026-01-15 07:03:00 CET,2026-01-15 17:48:00 CET,15m}]","""Holmenkollen""",10.663099,59.960214
"""NSR:StopPlace:6009""","""Subway""","[{""NSR:Quay:11027"",2026-01-15 07:03:00 CET,2026-01-15 17:59:00 CET,11m}, {""NSR:Quay:11026"",2026-01-15 07:08:00 CET,2026-01-15 17:57:00 CET,11m}]","""Carl Berners plass""",10.779682,59.925905
"""NSR:StopPlace:28079""","""Bus""","[{""NSR:Quay:48312"",2026-01-15 07:04:00 CET,2026-01-15 17:59:00 CET,8m}]","""Alkeveien""",5.60242,58.971783


In [8]:
parent_station_7_18.write_parquet("stasjoner_med_frekvens_10_15_7_18.parquet")
parent_station_7_20.write_parquet("stasjoner_med_frekvens_10_15_7_20.parquet")